In [ ]:
# Install pytorch with cuda

! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import logging
import torch

In [2]:
def get_device():
    if torch.cuda.is_available():
        logging.info("CUDA is available.")
        return torch.device("cuda")
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        logging.info("MPS is available.")
        return torch.device("mps")
    logging.info("CUDA and MPS are not available. Using CPU.")
    return torch.device("cpu")

In [3]:
get_device()

device(type='cuda')

In [4]:
class TransformersModel:
    def __init__(self, model_name: str):
        self.device = get_device()
        logging.info("Loading model and tokenizer...")
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        logging.info("Tokenizer loaded")
        
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.model.to(self.device)
        logging.info("Model loaded")
    
    def generate_response(self, text: str, max_length=32, num_beams=4):
        inputs = self.tokenizer.encode(text, return_tensors="pt").to(self.device)
        outputs = self.model.generate(inputs, max_length=max_length, num_beams=num_beams, early_stopping=True)
        response = self.tokenizer.decode(outputs[0].to("cpu"))
        return response.replace("<pad>", "").replace("<s>", "").replace("</s>", "").strip()

models = {
    "bigscience/T0_3B" : TransformersModel("bigscience/T0_3B"),
    "google/flan-t5-small" : TransformersModel("google/flan-t5-small"),
    "geektech/flan-t5-base-gpt4-relation" : TransformersModel("geektech/flan-t5-base-gpt4-relation"),
}

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\dunge\Desktop\Dev Workshop\development-workshop\.venv\lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
models["geektech/flan-t5-base-gpt4-relation"].generate_response("""You are supplied an article abstract formed as a JSON. Using the provided data, create a list of age groups which the article studied. DO NOT output any other text.
                                             
                                             "abstract": {
      "Background": "The UK prevalence of abdominal aortic aneurysm (AAA) is estimated at 4.9% in over 65‐year olds. Progressive and unpredictable enlargement can lead to rupture. Endovascular repair of AAAs involves a stent graft system being introduced via the femoral artery and manipulated within the aorta under radiological guidance. Following endograft deployment, a seal is formed at the proximal and distal landing zones to exclude the aneurysm sac from the circulation. With the increasing popularity of endovascular repair there has been an increase in the number of commercially available stent graft designs on the market. This is an update of the review first published in 2013.",
      "Objectives": "This review aimed to assess the different stent graft types for endovascular repair of AAA.",
      "Search methods": "The Cochrane Vascular Group Trials Search Co‐ordinator (TSC) searched the Specialised Register (last searched February 2015) and the Cochrane Register of Studies (2015, Issue 1). Trial databases were searched by the TSC for details of ongoing and unpublished studies.",
      "Selection criteria": "All published and unpublished randomised controlled trials (RCTs) of stent graft types in the repair of AAAs were sought without language restriction and in consultation with the Cochrane Vascular Group TSC.",
      "Data collection and analysis": "We planned to conduct data collection and analysis in accordance with the Cochrane Handbook for Systematic Reviews of Interventions .",
      "Main results": "No studies were identified that met the inclusion criteria. It was not possible to review the quality of the evidence in the absence of studies eligible for inclusion in the review.",
      "Authors' conclusions": "Unfortunately, no data exist regarding direct comparisons of the performance of different stent graft types. High quality randomised controlled trials evaluating stent graft types in abdominal endovascular aneurysm repair are required."
    }
                                             """, max_length=512, num_beams=10)

'Age groups studied include: <unk>'

In [28]:
models["geektech/flan-t5-base-gpt4-relation"].generate_response("""You are supplied an article abstract formed as a JSON. Using the provided data, create a comma separated list of article sections. Do not output any other text.
                                             
                                             "abstract": {
      "Background": "The UK prevalence of abdominal aortic aneurysm (AAA) is estimated at 4.9% in over 65‐year olds. Progressive and unpredictable enlargement can lead to rupture. Endovascular repair of AAAs involves a stent graft system being introduced via the femoral artery and manipulated within the aorta under radiological guidance. Following endograft deployment, a seal is formed at the proximal and distal landing zones to exclude the aneurysm sac from the circulation. With the increasing popularity of endovascular repair there has been an increase in the number of commercially available stent graft designs on the market. This is an update of the review first published in 2013.",
      "Objectives": "This review aimed to assess the different stent graft types for endovascular repair of AAA.",
      "Search methods": "The Cochrane Vascular Group Trials Search Co‐ordinator (TSC) searched the Specialised Register (last searched February 2015) and the Cochrane Register of Studies (2015, Issue 1). Trial databases were searched by the TSC for details of ongoing and unpublished studies.",
      "Selection criteria": "All published and unpublished randomised controlled trials (RCTs) of stent graft types in the repair of AAAs were sought without language restriction and in consultation with the Cochrane Vascular Group TSC.",
      "Data collection and analysis": "We planned to conduct data collection and analysis in accordance with the Cochrane Handbook for Systematic Reviews of Interventions .",
      "Main results": "No studies were identified that met the inclusion criteria. It was not possible to review the quality of the evidence in the absence of studies eligible for inclusion in the review.",
      "Authors' conclusions": "Unfortunately, no data exist regarding direct comparisons of the performance of different stent graft types. High quality randomised controlled trials evaluating stent graft types in abdominal endovascular aneurysm repair are required."
    }
                                             """)

'A review of the different stent graft types for endovascular repair of AAAs.'

In [5]:
models["geektech/flan-t5-base-gpt4-relation"].generate_response("""You are provided a background abstract from a study. 
                                                                The abstract is supplied below within the <abstract> HTML markers. 
                                                                Using the provided data, form a list of age groups which this abstract mentions. 
                                                                An example output is provided below as well. Do not output any other text.
                                                                
                                                                Example output: "Teens 10-16 years old, Adults 20-30 years old, Elderly 65> years old"
                                                                
                                                         <abstract>"The UK prevalence of abdominal aortic aneurysm (AAA) is estimated at 4.9% in over 65‐year olds. Progressive and unpredictable enlargement can lead to rupture. Endovascular repair of AAAs involves a stent graft system being introduced via the femoral artery and manipulated within the aorta under radiological guidance. Following endograft deployment, a seal is formed at the proximal and distal landing zones to exclude the aneurysm sac from the circulation. With the increasing popularity of endovascular repair there has been an increase in the number of commercially available stent graft designs on the market. This is an update of the review first published in 2013."</abstract>
       
                                                                """)

'Adults 20-30 years old, Adults 20-30 years old, Elderly 65> years old'